In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os,math
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path = 'covtype.csv'
classes = 7  # total classes
data=pd.read_csv(path, header=None)
X=data.iloc[:,:54]
Y=data.iloc[:,-1]
# scaling the inputs
X = MinMaxScaler().fit_transform(X)
# one hot encode output
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(581012, 54) (581012, 7)


In [3]:
# split in train-test as per requirenments
X_train,X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.483659)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(300000, 54) (300000, 7)
(281012, 54) (281012, 7)


In [4]:
############################ HALF ###############################
## this will extract hlaf of the features every time and join it with the past one. it stops when it gets min(1024,54*54).
n = X_train.shape[1]
dim = X_train.shape[1]
final_dim = min(32*32,n*n)
# convert 2-D to 3-D.
while dim<final_dim:
  features = int(dim/2)
  #f = feature-60
  model = Sequential()
  if (dim+features)>final_dim:
    features = final_dim - dim
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  else:
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=10,batch_size=256,verbose=1)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  X_train = np.concatenate((X_train, features1), axis=1) # concatenate on horizontal axis
  X_test = np.concatenate((X_test, features2), axis=1)
  dim = X_train.shape[1]
print(X_train.shape,X_test.shape)
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

Epoch 1/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.8270
Epoch 2/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6799
Epoch 3/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6519
Epoch 4/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6382
Epoch 5/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6291
Epoch 6/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6216
Epoch 7/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6156
Epoch 8/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6101
Epoch 9/10
1172/1172 [==============================] - 2s 1ms/step - loss: 0.6050
Epoch 10/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6008
Epoch 1/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.6938
Epoch 2/10
1172/1172 [==============================] - 1s 1ms/step - loss: 0.5973
Epo

In [7]:
# resize(reshape) it and convert it in 'rgb'
from PIL import Image
x_train = []
x_test = []
for i in range(len(X_train)):
    img = Image.fromarray(X_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(X_test)):
    img = Image.fromarray(X_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(300000, 32, 32, 3) (281012, 32, 32, 3)


In [9]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
1172/1172 [==============================] - 106s 90ms/step - loss: 0.3922 - accuracy: 0.8419 - val_loss: 0.3845 - val_accuracy: 0.8413
Epoch 2/20
1172/1172 [==============================] - 97s 83ms/step - loss: 0.3551 - accuracy: 0.8552 - val_loss: 0.3651 - val_accuracy: 0.8523
Epoch 3/20
1172/1172 [==============================] - 98s 84ms/step - loss: 0.3392 - accuracy: 0.8608 - val_loss: 0.4672 - val_accuracy: 0.8241
Epoch 4/20
1172/1172 [==============================] - 99s 85ms/step - loss: 0.3445 - accuracy: 0.8590 - val_loss: 0.3623 - val_accuracy: 0.8501
Epoch 5/20
1172/1172 [==============================] - 100s 86ms/step - loss: 0.3265 - accuracy: 0.8653 - val_loss: 0.3421 - val_accuracy: 0.8588
Epoch 6/20
1172/1172 [==============================] - 101s 86ms/step - loss: 0.3325 - accuracy: 0.8630 - val_loss: 0.3373 - val_accuracy: 0.8596
Epoch 7/20
1172/1172 [==============================] - 101s 86ms/step - loss: 0.3189 - accuracy: 0.8683 - val_loss: 0.33

In [10]:
# evaluate the model
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

9375/9375 [==============================] - 84s 9ms/step - loss: 0.2228 - accuracy: 0.9049
training accuracy:  0.9049333333969116
testing accuracy:  0.8942286968231201
